# Spark Pipeline on Kickstarter Pledge Dataset

## 1. Overview

### 1.1. Instructions

- **Choosing any sufficiently large open dataset** (less than 100000 lines are not allowed)


- **Choosing one variable to predict**


- **Implementing at least two supervised learning models**: classification, regression, recommender system, etc. Unsupervised tasks (e.g. clusterisation, associative rules, etc.) are not allowed


- **Mandatory use of Apache Spark** (e.g. on Google Cloud as we did during our lab sessions)


- A **full machine learning pipeline must be implemented**, which include:
    - Reading the data
    - Transforming data (extracting features, dealing with missing values if any, etc.)
    - Building models (build at least two models to compare)
    - Evaluating quality (use cross-validation or train/test split)

### 1.2. Dataset

### 1.3. Summary & Conclusion

The notebook was also ran locally using the installation steps for Spark described [here](https://sparkbyexamples.com/spark/spark-installation-on-linux-ubuntu/).

## 2. Environment Set-Up

We need the following libraries installed to set up the environment:

- kaggle (see documentation [here](https://github.com/Kaggle/kaggle-api#datasets))
- pyspark (see documentation [here](https://spark.apache.org/docs/latest/api/python/index.html))

In [ ]:
!pip install kaggle
!pip install pyspark

## 3. Dataset Download

In [ ]:
# Removes existing files that may have been downloaded locally
!rm -f kickstarter-projects.zip
!rm -f ks-projects-201612.csv ks-projects-201801.csv

In [ ]:
!ls
!mv home/qlr/kaggle.json /root/.kaggle/kaggle.json

In [ ]:
# Dowloads the raw dataset from the kaggle source
!kaggle datasets download -d kemical/kickstarter-projects

In [ ]:
# Unzips the raw dataset and keeps only the most recent instance
!unzip kickstarter-projects.zip
!rm -f ks-projects-201612.csv kickstarter-projects.zip
!ls

We only keep 'ks-projects-201801.csv', the most recent dataset available.

## 4. Library Imports & Spark Variables

In [14]:
from pyspark.context import SparkContext
from pyspark.sql.functions import unix_timestamp, ceil, isnan, when, count, col
from pyspark.sql.session import SparkSession
from pyspark.sql.types import FloatType

In [15]:
dataset_path = "/user/qlr/ks-projects-201801.csv"
dataset_format = "csv"
#spark_context = "local" #if run on a local computer
spark_context = "" #if run on Google Cloud

## 5. Starting a Spark Session & Loading the Dataset

In [ ]:
sc = SparkContext(spark_context)
spark = SparkSession(sc)

In [16]:
spark

In [17]:
campaigns = (spark
             .read
             .format(dataset_format)
             .options(header=True)
             .load(dataset_path))

## 6. Data Preprocessing

In [18]:
raw_columns_to_keep = [
    "ID","name","category","deadline","launched","country","usd_goal_real", #features
    "state" # target
]

replace_start_end_dates_with_duration = [
    "ID","name","category","total_duration","country","usd_goal_real", #features
    "state" # target
]

kept_columns_for_decision_tree = [
    "total_duration","usd_goal_real","category","country", #features
    "state" # target
]

deadline_format = "yyyy-MM-dd"
launched_format = "yyyy-MM-dd HH:mm:ss"

In [19]:
# Checks the type of each columns
campaigns.dtypes

[('ID', 'string'),
 ('name', 'string'),
 ('category', 'string'),
 ('main_category', 'string'),
 ('currency', 'string'),
 ('deadline', 'string'),
 ('goal', 'string'),
 ('launched', 'string'),
 ('pledged', 'string'),
 ('state', 'string'),
 ('backers', 'string'),
 ('country', 'string'),
 ('usd pledged', 'string'),
 ('usd_pledged_real', 'string'),
 ('usd_goal_real', 'string')]

In [20]:
# Checks for N/A
campaigns.select([count(when(isnan(c), c)).alias(c) for c in campaigns.columns]).show()

+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+----------------+-------------+
| ID|name|category|main_category|currency|deadline|goal|launched|pledged|state|backers|country|usd pledged|usd_pledged_real|usd_goal_real|
+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+----------------+-------------+
|  0|   0|       0|            0|       0|       0|   0|       0|      0|    0|      0|      0|          0|               0|            0|
+---+----+--------+-------------+--------+--------+----+--------+-------+-----+-------+-------+-----------+----------------+-------------+



In [21]:
# Keeps only the relevant columns
campaigns = campaigns.select(raw_columns_to_keep)
# --
campaigns.show(n=5)

+----------+--------------------+--------------+----------+-------------------+-------+-------------+--------+
|        ID|                name|      category|  deadline|           launched|country|usd_goal_real|   state|
+----------+--------------------+--------------+----------+-------------------+-------+-------------+--------+
|1000002330|The Songs of Adel...|        Poetry|2015-10-09|2015-08-11 12:12:28|     GB|      1533.95|  failed|
|1000003930|Greeting From Ear...|Narrative Film|2017-11-01|2017-09-02 04:43:57|     US|     30000.00|  failed|
|1000004038|      Where is Hank?|Narrative Film|2013-02-26|2013-01-12 00:20:50|     US|     45000.00|  failed|
|1000007540|ToshiCapital Reko...|         Music|2012-04-16|2012-03-17 03:24:11|     US|      5000.00|  failed|
|1000011046|Community Film Pr...|  Film & Video|2015-08-29|2015-07-04 08:35:03|     US|     19500.00|canceled|
+----------+--------------------+--------------+----------+-------------------+-------+-------------+--------+
o

In [22]:
# Computes a duration time (in day) from the launch and deadline features before dropping them
launch_times = unix_timestamp('launched', format = launched_format)
deadline_times = unix_timestamp('deadline', format = deadline_format)
time_difference = deadline_times - launch_times
campaigns = campaigns.\
    withColumn("total_duration",ceil(time_difference/(3600*24))).\
    select(replace_start_end_dates_with_duration)
# --
campaigns.show(n=5)

+----------+--------------------+--------------+--------------+-------+-------------+--------+
|        ID|                name|      category|total_duration|country|usd_goal_real|   state|
+----------+--------------------+--------------+--------------+-------+-------------+--------+
|1000002330|The Songs of Adel...|        Poetry|            59|     GB|      1533.95|  failed|
|1000003930|Greeting From Ear...|Narrative Film|            60|     US|     30000.00|  failed|
|1000004038|      Where is Hank?|Narrative Film|            45|     US|     45000.00|  failed|
|1000007540|ToshiCapital Reko...|         Music|            30|     US|      5000.00|  failed|
|1000011046|Community Film Pr...|  Film & Video|            56|     US|     19500.00|canceled|
+----------+--------------------+--------------+--------------+-------+-------------+--------+
only showing top 5 rows



In [ ]:
campaigns = campaigns.dropna()
for column in campaigns.columns:
    campaigns = campaigns.where(col(column).isNotNull())
campaigns = campaigns.dropDuplicates()
for condition in ['state!="undefined"', 'state!="live"']:
    campaigns = campaigns.where(condition)
campaigns.select("state").groupBy('state').count().orderBy(col("count").desc()).show()

In [23]:
campaigns = campaigns.dropna()
for column in campaigns.columns:
    campaigns = campaigns.where(col(column).isNotNull())
campaigns = campaigns.dropDuplicates()
for condition in ['state!="undefined"', 'state!="live"']:
    campaigns = campaigns.where(condition)
campaigns = campaigns.\
    withColumn("state",when(col("state") == "canceled", "failed").\
    when(col("state") == "suspended", "failed").\
    when(col("state") == "failed", "failed").\
    otherwise("successful"))
campaigns.select("state").groupBy('state').count().orderBy(col("count").desc()).show()

+----------+------+
|     state| count|
+----------+------+
|    failed|237579|
|successful|133429|
+----------+------+



In [24]:
campaigns.count()

371008

In [25]:
# Casts the relevant column(s) to their end types
for column in ["total_duration", "usd_goal_real"]:
    campaigns = campaigns.withColumn(column,col(column).cast(FloatType()))

### 6.1. Exploring the dataset

### 6.2. Building a dataset for a decision tree classifier

In [26]:
# Removes ID and name from the dataset
decision_tree_dataset = campaigns.select(kept_columns_for_decision_tree)
# --
decision_tree_dataset.show(n=5)
campaigns.dtypes

+--------------+-------------+-----------+-------+----------+
|total_duration|usd_goal_real|   category|country|     state|
+--------------+-------------+-----------+-------+----------+
|          29.0|       5000.0|Periodicals|     US|successful|
|          17.0|        300.0|  Sculpture|     US|successful|
|          30.0|      60000.0|Documentary|     US|    failed|
|          30.0|     222200.0| Television|     US|    failed|
|          30.0|      10000.0|        Pop|     US|successful|
+--------------+-------------+-----------+-------+----------+
only showing top 5 rows



[('ID', 'string'),
 ('name', 'string'),
 ('category', 'string'),
 ('total_duration', 'float'),
 ('country', 'string'),
 ('usd_goal_real', 'float'),
 ('state', 'string')]

## 7. Running a DecisionTreeClassifier Pipeline

In [27]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator

### 7.1. Creating a data processing pipeline

We will rely on indexing and assembling our data pipeline using the following stages:
- **StringIndexer** for all categorical columns
- **OneHotEncoder** for all categorical index columns
- **VectorAssembler** for all feature columns into one vector column

In [28]:
# Declares hyperparameters
training_size = 0.8
validation_size = 0.2
max_depth_grid = list(range(2,10))

In [29]:
# Creates a pipeline stage to string index each categorical feature column, and the label column
categorical_feature_columns = decision_tree_dataset.columns[2:4]
string_indexing_feature_columns = [StringIndexer(inputCol=column, 
                                                 outputCol='strindexed_' + column,
                                                 handleInvalid="skip")
                                   for column in categorical_feature_columns]
string_indexing_label_column = [StringIndexer(inputCol='state', 
                                              outputCol='label',
                                              handleInvalid="skip")]

In [30]:
# Creates a pipeline stage to one-hot encode each categorical feature column
onehot_encoding_feature_columns = [OneHotEncoder(inputCol='strindexed_' + column, 
                                                 outputCol='onehot_' + column) 
                                  for column in categorical_feature_columns]

In [31]:
# Creates a pipeline stage to vector assemble each categorical feature column
processed_feature_columns = list(map(lambda col_name: "onehot_" + col_name, categorical_feature_columns))
processed_feature_columns += ["total_duration", "usd_goal_real"]
vectorassembler_stage = VectorAssembler(inputCols=processed_feature_columns, 
                                        outputCol='features')

In [32]:
# Assembles the data processing pipeline
data_processing_pipeline = Pipeline(
    stages = string_indexing_feature_columns +
    string_indexing_label_column + 
    onehot_encoding_feature_columns + 
    [vectorassembler_stage]
)

In [33]:
# Fits the data processing pipeline
pipeline_model = data_processing_pipeline.fit(decision_tree_dataset)

In [34]:
final_columns = processed_feature_columns + ['features', 'label']
decision_tree_dataset_prepped = pipeline_model.transform(decision_tree_dataset.na.drop()).select(final_columns)
# --       
decision_tree_dataset_prepped.show(5)

+----------------+--------------+--------------+-------------+--------------------+-----+
| onehot_category|onehot_country|total_duration|usd_goal_real|            features|label|
+----------------+--------------+--------------+-------------+--------------------+-----+
|(158,[65],[1.0])|(22,[0],[1.0])|          29.0|       5000.0|(182,[65,158,180,...|  1.0|
|(158,[55],[1.0])|(22,[0],[1.0])|          17.0|        300.0|(182,[55,158,180,...|  1.0|
| (158,[1],[1.0])|(22,[0],[1.0])|          30.0|      60000.0|(182,[1,158,180,1...|  0.0|
|(158,[75],[1.0])|(22,[0],[1.0])|          30.0|     222200.0|(182,[75,158,180,...|  0.0|
|(158,[30],[1.0])|(22,[0],[1.0])|          30.0|      10000.0|(182,[30,158,180,...|  1.0|
+----------------+--------------+--------------+-------------+--------------------+-----+
only showing top 5 rows



In [35]:
# Checks for N/A
decision_tree_dataset_prepped.na.drop().show()

+-----------------+--------------+--------------+-------------+--------------------+-----+
|  onehot_category|onehot_country|total_duration|usd_goal_real|            features|label|
+-----------------+--------------+--------------+-------------+--------------------+-----+
|  (158,[4],[1.0])|(22,[0],[1.0])|          30.0|       6500.0|(182,[4,158,180,1...|  1.0|
|  (158,[3],[1.0])|(22,[0],[1.0])|          40.0|       1200.0|(182,[3,158,180,1...|  0.0|
|(158,[101],[1.0])|(22,[0],[1.0])|          25.0|        400.0|(182,[101,158,180...|  1.0|
|  (158,[2],[1.0])|(22,[0],[1.0])|           9.0|       3219.0|(182,[2,158,180,1...|  1.0|
| (158,[12],[1.0])|(22,[2],[1.0])|          45.0|      3701.24|(182,[12,160,180,...|  1.0|
|  (158,[4],[1.0])|(22,[0],[1.0])|          30.0|       7500.0|(182,[4,158,180,1...|  1.0|
| (158,[50],[1.0])|(22,[1],[1.0])|          30.0|       166.33|(182,[50,159,180,...|  0.0|
|  (158,[0],[1.0])|(22,[0],[1.0])|          27.0|       1999.0|(182,[0,158,180,1...|  1.0|

In [36]:
# Splits the dataset between training and validation sets
training, validation = decision_tree_dataset_prepped.randomSplit([training_size, validation_size], seed=0)

### 7.2. Creating a model pipeline

In [37]:
# Builds the estimator
dt = DecisionTreeClassifier(featuresCol='features', labelCol='label')

In [38]:
# Builds a parameter grid
param_grid = ParamGridBuilder().\
    addGrid(dt.maxDepth, max_depth_grid).\
    build()

In [39]:
# Builds the evaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")

In [40]:
# Builds the cross-validation model
cv = CrossValidator(estimator=dt, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=4)

In [ ]:
# Fits the cross-validation model
cv_model = cv.fit(decision_tree_dataset_prepped.na.drop())

In [ ]:
# Predicts on training data
show_columns = ['features', 'label', 'prediction', 'rawPrediction', 'probability']
pred_training_cv = cv_model.transform(training)
pred_training_cv.select(show_columns).show(5)

+--------------------+-----+----------+-----------------+--------------------+
|            features|label|prediction|    rawPrediction|         probability|
+--------------------+-----+----------+-----------------+--------------------+
|(182,[0,158,180,1...|  0.0|       1.0|[24479.0,28377.0]|[0.46312622975631...|
|(182,[0,158,180,1...|  1.0|       1.0|[24479.0,28377.0]|[0.46312622975631...|
|(182,[0,158,180,1...|  0.0|       1.0|[24479.0,28377.0]|[0.46312622975631...|
|(182,[0,158,180,1...|  0.0|       0.0|[69694.0,28701.0]|[0.70830834900147...|
|(182,[0,158,180,1...|  1.0|       1.0|[24479.0,28377.0]|[0.46312622975631...|
+--------------------+-----+----------+-----------------+--------------------+
only showing top 5 rows



In [ ]:
# Predicts on test data
pred_test_cv = cv_model.transform(validation)
pred_test_cv.select(show_columns).show(5)

+--------------------+-----+----------+------------------+--------------------+
|            features|label|prediction|     rawPrediction|         probability|
+--------------------+-----+----------+------------------+--------------------+
|(182,[0,158,180,1...|  1.0|       1.0| [24479.0,28377.0]|[0.46312622975631...|
|(182,[0,158,180,1...|  0.0|       1.0| [24479.0,28377.0]|[0.46312622975631...|
|(182,[0,158,180,1...|  1.0|       1.0| [24479.0,28377.0]|[0.46312622975631...|
|(182,[0,158,180,1...|  0.0|       0.0|[104275.0,70239.0]|[0.59751653162496...|
|(182,[0,158,180,1...|  0.0|       0.0| [69694.0,28701.0]|[0.70830834900147...|
+--------------------+-----+----------+------------------+--------------------+
only showing top 5 rows



In [ ]:
# Provides a confusion matrix
label_and_pred = cv_model.transform(decision_tree_dataset_prepped).select('label', 'prediction')
label_and_pred.rdd.zipWithIndex().countByKey()

defaultdict(int,
            {Row(label=0.0, prediction=0.0): 213100,
             Row(label=0.0, prediction=1.0): 24479,
             Row(label=1.0, prediction=0.0): 105052,
             Row(label=1.0, prediction=1.0): 28377})

In [ ]:
print('The best MaxDepth is:', cv_model.bestModel._java_obj.getMaxDepth())

('The best MaxDepth is:', 2)


In [47]:
print("The dataset contains " + str(campaigns.count()) + " rows.")

The dataset contains 371008 rows.
